# Filtering dataframe for fertility analysis

Edits are performed on dataset "fertilityDF_W61.csv" and "fertilityDF_W68.csv" i.e. after "BuildingDataframe.ipynb" & "FertilityTraits.ipynb" have been run. The code generates a clean dataset for further analysis in:
- "../Gigacow-tools/Projects/HeatStressEvaluation/DataPreprocessing/PrepCRThresholdAnalysis.ipynb": for THI threshold estimation using piecewise regression 
- "../Gigacow-tools/HeatStressEvaluation/Modeling/FertilityAnalysis.ipynb": for analysis of classical fertility traits

Filtering steps performed:
- Add 305d MY from raw data and check missing MY records (a756bc39, 6d38bc90)
- Keep only data from SH and NRDC

The following filtering steps are according to NAV (2024):
- Keep only lactation 1-8 
- Make parity 1, 2, >=3 
- Records within 150 days from data extraction are excluded from the data set 
- Only the first 10 inseminations are accepted for CR 
- Age at first calving: 550d - 1100d 
    - Note: risk losing cows due to missing contract from cow database
- CI maximum 2 years for cows
- CFS 20 - 230d
- FLS max 365d

In addition, putting thresholds on classical fertility traits according to NAV or µ+-2SD of respective trait
- CFI: 20-230
- CLI: 20-217
- FLI: 0-365
- CI: 301-730
- GL: 260-302

Allow for minimum 5 records in HYS groups (of insemination date) to help convergence in analysis

Allow for minimum 5 cows of the same breed in contemporary groups (HYS of ins)

Add threshold for 305d MY (7,395 - 16,255kg)

Basic descriptive analysis at end of script

This script generates a full dataframe ("fertilityDF_W_MY_filtered.csv") and a clean dataset called "fertility_filtered.csv" containing the following columns:
- SE_Number (ID identification)
- Breed 
    * NRDC: SRB, DR, SAB
    * SH
- LactationNumber (number of calvings)
- Parity (lactation 1,2,>=3)
- InseminationDate (Date of insemination)
- HYS of insemination
- HeatStress (whether the cow experienced HS 7d prior to, during and 7d after insemination)
- Milk_Kg (305d)
- CFI (Interval between calving and first insemination)
- CLI (Interval between calving and last insemination)
- FLI (Interval between first and last insemination)
- NINS (Total number of inseminations during service period)
- CR0 (Conception rate)
- CI (Calving interval)
- GL (Gestation length)

In [1]:
from datetime import timedelta
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob

from fpdf import FPDF

import statsmodels.api as sm
import statsmodels.formula.api as smf
from itertools import product

In [ ]:
# LOAD DATA
# df3 = pd.read_csv("../Data/MergedData/fertilityDF_W61.csv", low_memory=False)
df3 = pd.read_csv("../Data/MergedData/fertilityDF_W68.csv", low_memory=False)
df3

In [ ]:
# Crude data distribution
print(f"No. observations in fertilityDF_W68.csv.csv: {df3.shape}")
test = df3.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate", "PregnancyCheckDate"])
print(f"Double check no. pregnancy checks in fertilityDF_W.csv: {test.shape}")
print(f'No. ins without pregnancy checks: {df3.shape[0] - test.shape[0]}')

test = df3.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate"])
print(f"No. inseminations in fertilityDF_W.csv: {test.shape}")
test = df3.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. lactations in fertilityDF_W.csv: {test.shape}")
test = df3.drop_duplicates(subset=["SE_Number"])
print(f"No. cows in fertilityDF_W.csv: {test.shape}")

## Keep only herds with (MY_W) data

In [ ]:
# Count number of inseminations in herds
ins_count = df3.groupby("FarmName_Pseudo")["InseminationDate"].nunique()
print(ins_count)

In [ ]:
# Count presence of HeatStress in the herds with issues with MY data
herds_to_keep = ['a756bc39', '6d38bc90']
df4 = df3[df3["FarmName_Pseudo"].isin(herds_to_keep)]

non_nan_count = df4['HS'].notna().sum()
print(f"Number of non-NaN values: {non_nan_count}")

is_nan_count = df4['HS'].isna().sum()
print(f"Number of NaN values: {is_nan_count}")

In [ ]:
# Check if herds with issues with MY recording in DelPro have raw data from cow database
MY = pd.read_csv("C:/Users/pagd0001/Desktop/Gigacow/Data/20241009/Gigacow-tools/Projects/HeatStressEvaluation/Data/CowData/Kok_LactationReturn240820.csv", low_memory=False, delimiter=";")
col_keep = ["BirthID", "CalvingNumber", "Milk_Kg"]
MY = MY[col_keep]
MY.rename(columns={"BirthID": "SE_Number", "CalvingNumber": "LactationNumber"}, inplace=True)
MY = MY.drop_duplicates(subset=["SE_Number", "LactationNumber", "Milk_Kg"])
MY

In [ ]:
df3

In [ ]:
# Add 305d MY records to fertilityDF_W.csv
df3 = pd.merge(df3, MY, on=["SE_Number", "LactationNumber"], how="left")

# df3.to_csv("../Data/MergedData/fertilityDF_W61.csv", index=False)
df3.to_csv("../Data/MergedData/fertilityDF_W68.csv", index=False)
df3

In [ ]:
# Count presence of 305d MY records in the herds with issues with MY data in DelPro
herds_to_keep = ['a756bc39', '6d38bc90']
df4 = df3[df3["FarmName_Pseudo"].isin(herds_to_keep)]

non_nan_count = df4['Milk_Kg'].notna().sum()
print(f"Number of non-NaN values: {non_nan_count}")

is_nan_count = df4['Milk_Kg'].isna().sum()
print(f"Number of NaN values: {is_nan_count}")

In [ ]:
""" =========================================================================>>> These herds have data reported to the cow database and can be included!
# Remove herds with missing MY records (a756bc39, 6d38bc90)
# Herds to remove
herds_to_remove = ['a756bc39', '6d38bc90']
df3 = df3[~df3['FarmName_Pseudo'].isin(herds_to_remove)]
"""

In [ ]:
print(f"No. observations in fertilityDF_W68.csv.csv: {df3.shape}")
test = df3.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate", "PregnancyCheckDate"])
print(f"Double check no. pregnancy checks in fertilityDF_W.csv: {test.shape}")
print(f'No. ins without pregnancy checks: {df3.shape[0] - test.shape[0]}')

test = df3.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate"])
print(f"No. inseminations in fertilityDF_W.csv: {test.shape}")
test = df3.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. lactations in fertilityDF_W.csv: {test.shape}")
test = df3.drop_duplicates(subset=["SE_Number"])
print(f"No. cows in fertilityDF_W.csv: {test.shape}")

## Data from SH and NRDC

In [21]:
# Load data
# df3 = pd.read_csv("../Data/MergedData/fertilityDF_W61.csv", low_memory=False)
df3 = pd.read_csv("../Data/MergedData/fertilityDF_W68.csv", low_memory=False)

In [22]:
# Keep only data from SH and NRDC
breeds_to_keep = ["NRDC", "SLB"]
df3 = df3[df3["Breed"].isin(breeds_to_keep)]

In [ ]:
print(f"No. observations in file: {df3.shape}")
test = df3.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate", "PregnancyCheckDate"])
print(f"Double check no. pregnancy checks in file: {test.shape}")
print(f'No. ins without pregnancy checks: {df3.shape[0] - test.shape[0]}')

test = df3.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate"])
print(f"No. inseminations in file: {test.shape}")
test = df3.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. lactations in file: {test.shape}")
test = df3.drop_duplicates(subset=["SE_Number"])
print(f"No. cows in file: {test.shape}")

## Allow for maximum lactation 1-8

In [ ]:
# Distribution of lactations
check = df3.drop_duplicates(subset=["SE_Number", "LactationNumber"])
value_counts = check['LactationNumber'].value_counts()
print(value_counts)

In [ ]:
# Keep only lactation 1-8
df3 = df3[df3["LactationNumber"] <= 8]

# No. lactations and cows in data
print(f"No. observations in file: {df3.shape}")
test = df3.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate", "PregnancyCheckDate"])
print(f"Double check no. pregnancy checks in file: {test.shape}")
print(f'No. ins without pregnancy checks: {df3.shape[0] - test.shape[0]}')

test = df3.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate"])
print(f"No. inseminations in file: {test.shape}")
test = df3.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. lactations in file: {test.shape}")
test = df3.drop_duplicates(subset=["SE_Number"])
print(f"No. cows in file: {test.shape}")

In [26]:
# Make Parity 1-3
df3["Parity"] = df3["LactationNumber"]
df3.loc[(df3['LactationNumber'] >= 3) & (df3['LactationNumber'] <= 8), 'Parity'] = 3

## Records within 150 days from data extraction are excluded from the data set (i.e. open records)

In [ ]:
# Find open records
df3["extraction_limit"] = pd.to_datetime(df3["extraction_limit"])
df3["InseminationDate"] = pd.to_datetime(df3["InseminationDate"])

df3["open_rec"] = (df3["extraction_limit"] - df3["InseminationDate"]).dt.days

# Plot histogram
plt.hist(df3['open_rec'], bins=50, color='blue', edgecolor='black')
plt.xlabel('date')
plt.ylabel('Frequency')
plt.show()

In [ ]:
non_nan_count = df3['open_rec'].notna().sum()
print(f"Number of non-NaN values: {non_nan_count}")

is_nan_count = df3['open_rec'].isna().sum()
print(f"Number of NaN values: {is_nan_count}")

In [29]:
df3 = df3[(df3["open_rec"] > 150)]

In [ ]:
print(f"No. observations in file: {df3.shape}")
test = df3.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate", "PregnancyCheckDate"])
print(f"Double check no. pregnancy checks in file: {test.shape}")
print(f'No. ins without pregnancy checks: {df3.shape[0] - test.shape[0]}')

test = df3.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate"])
print(f"No. inseminations in file: {test.shape}")
test = df3.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. lactations in file: {test.shape}")
test = df3.drop_duplicates(subset=["SE_Number"])
print(f"No. cows in file: {test.shape}")

## Only keep first 10 ins

In [ ]:
# Only the first 10 inseminations are accepted
# Distribution of NINS
value_counts = df3["NINS"].value_counts()
print(value_counts)

In [ ]:
# Count ins
df4 = df3.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate"])
df4

In [ ]:
df3["InseminationDate"] = pd.to_datetime(df3["InseminationDate"])

df4 = df4.copy()
df4['InseminationDate'] = pd.to_datetime(df4['InseminationDate'])

df4['InsCount'] = df4.groupby(['SE_Number', 'LactationNumber']).cumcount() + 1
col_keep = ["SE_Number", "LactationNumber", "InseminationDate", "InsCount"]
df4 = df4[col_keep]

df5 = pd.merge(df3, df4, on=["SE_Number", "LactationNumber", "InseminationDate"], how="left")
df5

In [ ]:
# Distribution of NINS 
value_counts = df5["InsCount"].value_counts()
print(value_counts)

df5.to_csv("test.csv", index=False)

In [ ]:
# The first 10 ins are accepted
df5 = df5[df5['InsCount'] != 11]

value_counts = df5["InsCount"].value_counts()
print(value_counts)

## Age at first calving: 550d - 1100d 
- Note: risk losing cows due to missing contracts for cow database!

First merge calving data from cow database and delpro management system

In [ ]:
# Load raw data to get full calving history from cow database
dfkok = pd.read_csv("../Data/CowData/Kok_Calving240820.csv", delimiter=";", low_memory=False)
col_keep = ["BirthID", "CalvingDate", "CalvingNumber"]
dfkok = dfkok[col_keep]
dfkok = dfkok.rename(columns={"BirthID": "SE_Number", "CalvingNumber": "LactationNumberKok", "CalvingDate": "CalvingDateKok"})
dfkok = dfkok.drop_duplicates(subset=["SE_Number", "LactationNumberKok"])
dfkok

In [ ]:
# Also load calving dates from DelPro to keep as much data as possible from herds with missing cow database contracts
dfDelPro = pd.read_csv("../Data/CowData/dfDelPro.csv", low_memory=False)
col_keep = ["SE_Number", "LactationNumberDelPro", "CalvingDateDelPro"]
dfDelPro = dfDelPro[col_keep]
dfDelPro = dfDelPro.drop_duplicates(subset=["SE_Number", "LactationNumberDelPro"])
dfDelPro

In [ ]:
dfkok['CalvingDateKok'] = pd.to_datetime(dfkok['CalvingDateKok'])
dfDelPro['CalvingDateDelPro'] = pd.to_datetime(dfDelPro['CalvingDateDelPro'])

# Merge
merged_df = pd.merge(
    dfkok,
    dfDelPro,
    how='outer',
    left_on=['SE_Number', 'CalvingDateKok', 'LactationNumberKok'],
    right_on=['SE_Number', 'CalvingDateDelPro', 'LactationNumberDelPro']
)
merged_df

In [ ]:
# fillna to create combined lactation number and calving dates
merged_df["LactationNumber"] = merged_df["LactationNumberKok"].fillna(merged_df["LactationNumberDelPro"])
merged_df["CalvingDate"] = merged_df["CalvingDateKok"].fillna(merged_df["CalvingDateDelPro"])
merged_df = merged_df.drop(columns=["CalvingDateKok", "CalvingDateDelPro", "LactationNumberKok", "LactationNumberDelPro"])
merged_df

Now load birth date data

In [ ]:
# Load raw data to get BirthDate
bd = pd.read_csv("../Data/CowData/Kok_Lineage240821.csv", delimiter=";", low_memory=False)
col_keep = ["BirthID", "BirthDate"]
bd = bd[col_keep]
bd = bd.rename(columns={"BirthID": "SE_Number", "BirthDate": "BirthDateKok" })
bd = bd.drop_duplicates(subset=["SE_Number"])
bd

In [70]:
# Also load from DelPro to keep as many cows as possible from herds without cow database contract
bd_DP = pd.read_csv("../Data/CowData/Del_Cow240823.csv", delimiter=";", low_memory=False)
col_keep = ["SE_Number", "BirthDate"]
bd_DP = bd_DP[col_keep]
bd_DP.rename(columns={"BirthDate": "BirthDateDelPro"}, inplace=True)
bd_DP = bd_DP.drop_duplicates(subset=["SE_Number"])

In [ ]:
# Combine birth date data
bd['BirthDateKok'] = pd.to_datetime(bd['BirthDateKok'])
bd_DP['BirthDateDelPro'] = pd.to_datetime(bd_DP['BirthDateDelPro'])

# Merge
merged_df2 = pd.merge(
    bd,
    bd_DP,
    how='outer',
    left_on=['SE_Number'],
    right_on=['SE_Number']
)
merged_df2

In [ ]:
# Use fillna to create a common BirthDate
merged_df2["BirthDate"] = merged_df2["BirthDateKok"].fillna(merged_df2["BirthDateDelPro"])
merged_df2 = merged_df2.drop(columns=["BirthDateKok", "BirthDateDelPro"])
merged_df2

Combine birth and calving data

In [ ]:
# Merge
merged_df = pd.merge(
    merged_df2,
    merged_df,
    how='outer',
    left_on=['SE_Number'],
    right_on=['SE_Number']
)

merged_df = merged_df[merged_df["SE_Number"].notna()]
merged_df

In [ ]:
# Remove "Unknown" cow
merged_df = merged_df.loc[merged_df['SE_Number'] != 'Unknown']
merged_df

In [ ]:
# Check missing values
# Here a lot of missing values from calving data is expected due to missing contract from certain herds
missing_BD = merged_df['BirthDate'].isna().sum()
print(f"No. missing birth dates: {missing_BD}")

missing_CD = merged_df['CalvingDate'].isna().sum()
print(f"No. missing calving dates: {missing_CD}")

Calculate age at first calving

In [ ]:
# Keep only data from first lactation
merged_df = merged_df[merged_df["LactationNumber"] == 1]
merged_df = merged_df.copy()

# Change date format
merged_df["BirthDate"] = pd.to_datetime(merged_df["BirthDate"])
merged_df["CalvingDate"] = pd.to_datetime(merged_df["CalvingDate"])

# Make AFC
merged_df["AgeFirstCalving"] = (merged_df["CalvingDate"] - merged_df["BirthDate"]).dt.days

# Keep only these two columns
col_keep = ["SE_Number", "AgeFirstCalving"]
merged_df = merged_df[col_keep]
merged_df

In [ ]:
missing_AFC = merged_df['AgeFirstCalving'].isna().sum()
print(f"No. missing values AgeFirstCalving: {missing_AFC}")

In [ ]:
# Plot
plt.hist(merged_df['AgeFirstCalving'], bins=50, color='blue', edgecolor='black')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Merge back onto current dataset
df6 = pd.merge(df5, merged_df, on=["SE_Number"], how="left")
df6

In [ ]:
non_nan_count = df6['AgeFirstCalving'].notna().sum()
print(f"Number of non-NaN values: {non_nan_count}")

is_nan_count = df6['AgeFirstCalving'].isna().sum()
print(f"Number of NaN values: {is_nan_count}")

In [81]:
# Filter age at first calving keeping only inseminations from cows with an age between 550 and 1100days old
df6 = df6[df6['AgeFirstCalving'].between(550, 1100)]

In [ ]:
# Plot
plt.hist(df6['AgeFirstCalving'], bins=50, color='blue', edgecolor='black')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

In [ ]:
print(f"No. observations in file: {df6.shape}")

test = df6.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate"])
print(f"No. inseminations in file: {test.shape}")
test = df6.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. lactations in file: {test.shape}")
test = df6.drop_duplicates(subset=["SE_Number"])
print(f"No. cows in file: {test.shape}")

## Thresholds for classical fertility traits in editing full material (insemination wise)
- CI maximum 2 years for cows
- CFS 20 - 230d
- FLS max 365d

In [ ]:
# Look at distribution of CI, CFI and FLI
df_fert = df6.drop_duplicates(subset=["SE_Number", "LactationNumber"])

plt.hist(df_fert['CFI'], bins=50, color='blue', edgecolor='black')
plt.xlabel('CFI')
plt.ylabel('Frequency')
plt.show()

plt.hist(df_fert['CLI'], bins=50, color='blue', edgecolor='black')
plt.xlabel('CLI')
plt.ylabel('Frequency')
plt.show()

plt.hist(df_fert['FLI'], bins=50, color='blue', edgecolor='black')
plt.xlabel('FLI')
plt.ylabel('Frequency')
plt.show()

In [85]:
# Thresholds
# Filter for CI values less than or equal to 730 or NaN
df6 = df6[(df6["CI"] <= 730) | (df6["CI"].isna())]

# Filter for CFI values between 20 and 230 (inclusive) or NaN
df6 = df6[(df6["CFI"].between(20, 230, inclusive="both")) | (df6["CFI"].isna())]

# Filter for FLI values less than or equal to 365 or NaN
df6 = df6[(df6["FLI"] <= 365) | (df6["FLI"].isna())]

# df6.to_csv("../Data/MergedData/fertilityDF_W_MY61_filtered.csv", index=False)
df6.to_csv("../Data/MergedData/fertilityDF_W_MY68_filtered.csv", index=False)

In [ ]:
print(f"No. observations in fertilityDF_W_MY68_filtered.csv.csv: {df6.shape}")

test = df6.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate"])
print(f"No. inseminations in filtered data: {test.shape}")
test = df6.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. lactations in filtered data: {test.shape}")
test = df6.drop_duplicates(subset=["SE_Number"])
print(f"No. cows in filtered data: {test.shape}")

## Minimum and maximum threshold on specific classical fertility traits
- i.e. if not within range, put to missing

In [87]:
# Load unfiltered data to check distribution
# fert_df = pd.read_csv("../Data/MergedData/fertilityDF_W61.csv", low_memory=False)
fert_df = pd.read_csv("../Data/MergedData/fertilityDF_W68.csv", low_memory=False)

fert_df = fert_df.drop_duplicates(subset=["SE_Number", "LactationNumber"])

In [ ]:
# Look at distributions
summary_stats = fert_df['CI'].describe()
percentiles = np.percentile(fert_df['CI'].dropna(), [1, 5, 10, 90, 95, 99])

print("Descriptive Statistics:\n", summary_stats)
print("\nPercentiles (1%, 5%, 10%, 90%, 95%, 99%):", percentiles)

In [ ]:
plt.hist(fert_df['CFI'], bins=50, color='blue', edgecolor='black')
plt.xlabel('CFI')
plt.ylabel('Frequency')
plt.show()

In [92]:
# Put thresholds on traits: Define the valid range and set outside this range to missing
# df6 = pd.read_csv("../Data/MergedData/fertilityDF_W_MY61_filtered.csv", low_memory=False)
df6 = pd.read_csv("../Data/MergedData/fertilityDF_W_MY68_filtered.csv", low_memory=False)

# CFI: => 20-230d according to NAV
CFI_min = 20
CFI_max = 230

df6.loc[(df6["CFI"] < CFI_min) | (df6["CFI"] > CFI_max), "CFI"] = np.nan

# CLI: if CFI min 20 and FLI 0 then CLI also 20d. 104 +- 53 
CLI_min = 20
CLI_max = 210

df6.loc[(df6["CLI"] < CLI_min) | (df6["CLI"] > CLI_max), "CLI"] = np.nan

# FLI: 0-365d according to NAV
FLI_min = 0
FLI_max = 365

df6.loc[(df6["FLI"] < FLI_min) | (df6["FLI"] > FLI_max), "FLI"] = np.nan

# CI: 306-730d, 730d according to NAV. 372 +- 66
CI_min = 306
CI_max = 730

df6.loc[(df6["CI"] < CI_min) | (df6["CI"] > CI_max), "CI"] = np.nan

# GL: 260 - 302 according to NAV. 
GL_min = 260
GL_max = 302

df6.loc[(df6["GL"] < GL_min) | (df6["GL"] > GL_max), "GL"] = np.nan

In [ ]:
# Double check distributions after handling outliers
summary_stats = df6['CFI'].describe()
percentiles = np.percentile(df6['CFI'].dropna(), [1, 5, 10, 90, 95, 99])

print("Descriptive Statistics:\n", summary_stats)
print("\nPercentiles (1%, 5%, 10%, 90%, 95%, 99%):", percentiles)

## Allow for minimum 5 records in HYS groups (of insemination date)

In [ ]:
# Make HYS of insemination
df6['InseminationDate'] = pd.to_datetime(df6['InseminationDate'])


# Function to determine the season based on month
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 1
    elif month in [3, 4, 5]:
        return 2
    elif month in [6, 7, 8]:
        return 3
    elif month in [9, 10, 11]:
        return 4


# Apply the function to create a 'YearSeason' variable
df6['YearSeason'] = df6['InseminationDate'].apply(lambda x: f"{x.year}0{get_season(x)}")

# Make HYS
df6["HYS"] = df6["FarmName_Pseudo"] + df6["YearSeason"]
df6 

In [ ]:
# Check distribution of HYS
print(df6["HYS"].value_counts())

In [ ]:
# HYS requiring minimum 5 events, otherwise remove
# Count occurrences of each HYS group and filter to keep only 5 or above
hys_counts = df6["HYS"].value_counts()
valid_hys = hys_counts[hys_counts >= 5].index
df6 = df6[df6["HYS"].isin(valid_hys)]

print(df6["HYS"].value_counts())

## Require minimum 5 cows within the same breed and in the same contemporary group HYS

In [97]:
# Group by HerdYearSeason and Breed, and filter for groups with at least 5 cows
df6 = (
    df6.groupby(['HYS', 'Breed'])
    .filter(lambda x: len(x) >= 5)  # Keep groups with at least 5 cows
)

## Add threshold for 305d MY
- unfortunately missing a lot of data due to missing contracts

In [ ]:
# Occurrence of MY data
non_nan_count = df6['Milk_Kg'].notna().sum()
print(f"Number of non-NaN values: {non_nan_count}")

is_nan_count = df6['Milk_Kg'].isna().sum()
print(f"Number of NaN values: {is_nan_count}")

In [ ]:
# Plot MY
plt.hist(df6['Milk_Kg'], bins=50, color='blue', edgecolor='black')
plt.xlabel('MY')
plt.ylabel('Frequency')
plt.show()

# Look at distribution
summary_stats = df6['Milk_Kg'].describe()
percentiles = np.percentile(df6['Milk_Kg'].dropna(), [1, 5, 10, 90, 95, 99])

print("Descriptive Statistics:\n", summary_stats)
print("\nPercentiles (1%, 5%, 10%, 90%, 95%, 99%):", percentiles)

In [100]:
# µ: 11,753. SD: 2197 => Thresholds: 7,359 - 16,147
MY_min = 7359
MY_max = 16147

df6.loc[(df6["Milk_Kg"] < MY_min) | (df6["Milk_Kg"] > MY_max), "Milk_Kg"] = np.nan

In [ ]:
plt.hist(df6['Milk_Kg'], bins=50, color='blue', edgecolor='black')
plt.xlabel('MY')
plt.ylabel('Frequency')
plt.show()

In [102]:
# Remove missing StartDates which were added form merge above
df6 = df6[df6["StartDate"].notna()]
print(df6.shape)

# Save full filtered data - note with THI in 7d window around inseminations
# df6.to_csv("../Data/MergedData/fertilityDF_W_MY61_filtered.csv", index=False)
df6.to_csv("../Data/MergedData/fertilityDF_W_MY68_filtered.csv", index=False)

## Save filtered dataset

In [107]:
# Clean dataset (probably added from merged in age at calving variables)
Ins_start = "2022-01-01"
Ins_stop = "2024-08-18"

df7 = df6[(df6['InseminationDate'] >= Ins_start) & (df6['InseminationDate'] <= Ins_stop)]

col_keep = ["FarmName_Pseudo", "SE_Number", "Breed", "LactationNumber", "Parity", "InseminationDate", "HYS", "HS", "THI_3D_Above_68", "HS_sum",
            "Milk_Kg", "CFI", "CLI", "FLI", "NINS", "CR0", "CI", "GL"]
df7 = df7[col_keep]
df7 = df7.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate"])
# df7.to_csv("../Data/MergedData/fertility_filtered61.csv", index=False)
df7.to_csv("../Data/MergedData/fertility_filtered68.csv", index=False)

In [ ]:
df7

## Check how much data we have that will be used in further analysis

In [ ]:
# Iterate through all columns
for column in df7.columns:
    # Count non-NaN values
    non_nan_count = df7[column].notna().sum()
    # Count NaN values
    is_nan_count = df7[column].isna().sum()
    # Print results
    print(f"Column: {column}")
    print(f"No. of non-NaN values: {non_nan_count}")
    print(f"No. of NaN values: {is_nan_count}")
    print()

## Remove missing milk yield
i.e. this removes herds where we are lacking cow database contracts

In [ ]:
df8 = df7[df7["Milk_Kg"].notna()]
print(df8.shape)

df_lact = df8.drop_duplicates(subset=["FarmName_Pseudo"])
col_keep = ["FarmName_Pseudo"]
df_lact = df_lact[col_keep]
print(df_lact.shape)
print(f"Herds in filtered data: \n", df_lact.to_string(index=False))

## Add DIM at insemination for modelling

In [ ]:
df = pd.read_csv("../Data/MergedData/fertilityDF_W_MY68_filtered.csv", low_memory=False)

df["InseminationDate"] = pd.to_datetime(df["InseminationDate"])
df["CalvingDate"] = pd.to_datetime(df["CalvingDate"])
df["DIMatINS"] = (df["InseminationDate"] - df["CalvingDate"]).dt.days + 1
df

In [ ]:
# Save only one obs per ins, and drop unnecessary columns
col_keep = ["SE_Number", "LactationNumber", "InseminationDate", "DIMatINS"]
df = df[col_keep]
df = df.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate"])

# Merge DIMatINS onto working df
df8 = pd.merge(df8, df, on=["SE_Number", "LactationNumber", "InseminationDate"], how="left")
df8

In [122]:
# Save df
df8.to_csv("../Data/MergedData/fertility_filtered68.csv", index=False)

# Descriptive statistics - filtered data

In [ ]:
# Filtered df
df_lact = pd.read_csv("../Data/MergedData/fertility_filtered68.csv", low_memory=False)

count_my_rec = df_lact.groupby(["Parity"])["InseminationDate"].count().reset_index()
print(f"No. of insemination records divided over parities: \n", count_my_rec.to_string(index=False))

count_my_rec = df_lact.groupby(["Parity", "Breed"])["InseminationDate"].count().reset_index()
print(f"No. of insemination records divided over parities and breeds: \n", count_my_rec.to_string(index=False))

# By parity
for_my_rec5 = df_lact.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. of parities in file: {for_my_rec5.shape}")

count_my_rec = for_my_rec5.groupby(["Breed"])["SE_Number"].count().reset_index()
print(f"No. of lactations from NRDC and SH: \n", count_my_rec.to_string(index=False))  

count_my_rec = for_my_rec5.groupby(["Parity", "Breed"])["SE_Number"].count().reset_index()
print(f"No. of cows within breed and parity: \n", count_my_rec.to_string(index=False))

# By cows
for_my_rec4 = df_lact.drop_duplicates(subset=["SE_Number"])
print(f"No. of cows in file: {for_my_rec4.shape}") 

for_my_rec5 = for_my_rec4.drop_duplicates(subset=["SE_Number"])
count_my_rec = for_my_rec5.groupby(["Breed"])["SE_Number"].count().reset_index()
print(f"No. of cows from NRDC, SH: \n", count_my_rec.to_string(index=False))

# Herd info
df_lact = pd.read_csv("../Data/MergedData/fertility_filtered68.csv", low_memory=False)
df_lact = df_lact.drop_duplicates(subset=["FarmName_Pseudo"])
col_keep = ["FarmName_Pseudo"]
df_lact = df_lact[col_keep]
print(df_lact.shape)
print(f"Herds in filtered data: \n", df_lact.to_string(index=False))